In [1]:
import sqlite3
import pandas as pd

# Load the CSV files into pandas DataFrames
sales_df = pd.read_csv("sales.csv")
categories_df = pd.read_csv("product_categories.csv")


In [2]:
# Create (or connect to) a local SQLite database
conn = sqlite3.connect("sales_data.db")  # This will create the DB if it doesn't exist

# Write DataFrames to SQLite tables
sales_df.to_sql("Sales", conn, if_exists="replace", index=False)
categories_df.to_sql("ProductCategories", conn, if_exists="replace", index=False)


5

In [3]:
# Optional: Confirm tables exist
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name
0,Sales
1,ProductCategories


In [4]:
query = """
WITH RegionRevenue AS (
    SELECT Region, SUM(Revenue) AS TotalRevenue
    FROM Sales
    GROUP BY Region
),
AboveAverageRegions AS (
    SELECT Region
    FROM RegionRevenue
    WHERE TotalRevenue > (SELECT AVG(TotalRevenue) FROM RegionRevenue)
)
SELECT 
    S.Region,
    P.Category,
    SUM(S.Revenue) AS TotalRevenue
FROM Sales S
JOIN ProductCategories P ON S.ProductID = P.ProductID
WHERE S.Region IN (SELECT Region FROM AboveAverageRegions)
GROUP BY S.Region, P.Category
ORDER BY S.Region, TotalRevenue DESC;
"""

result_df = pd.read_sql(query, conn)
result_df


,Region,Category,TotalRevenue
0,East,Apparel,1430
1,East,Electronics,900


In [5]:
# Create index on Revenue column
conn.execute("CREATE INDEX IF NOT EXISTS idx_sales_revenue ON Sales(Revenue);")
conn.commit()

# Confirm index was created
pd.read_sql("PRAGMA index_list('Sales');", conn)


,seq,name,unique,origin,partial
0,0,idx_sales_revenue,0,c,0


### Interpretation

The SQL query identifies product categories contributing to total revenue in regions that perform above the average. This allows sales teams to focus on successful segments and replicate success in other areas. Creating an index on the `Revenue` column improves query performance, especially with filters or sorting. Though our dataset is small, this optimization matters significantly with larger volumes, enabling faster business intelligence reporting and operational decisions.
